In [ ]:
! pip install feature-engine

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine import categorical_encoders as ce

In [ ]:
df=pd.read_csv("/content/drive/My Drive/car_project/final.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,variant,price,Year,Kms Driven,Fuel Type,Owner,Transmission,city,brand,model
0,Maruti Ertiga ZDI,"454,675",2012,"98,284",Diesel,1,Manual,Delhi-ncr,Maruti,Ertiga
1,Maruti Ciaz VXI PLUS,"658,520",2015,"11,600",Petrol,1,Manual,Delhi-ncr,Maruti,Ciaz
2,Maruti Swift VDI,"220,366",2011,"75,829",Diesel,1,Manual,Delhi-ncr,Maruti,Swift
3,Maruti Swift Dzire VXI 1.3,"257,084",2009,"75,753",Petrol,1,Manual,Delhi-ncr,Maruti,Swift
4,Volkswagen Polo COMFORTLINE 1.2L PETROL,"385,850",2014,"37,171",Petrol,1,Manual,Delhi-ncr,Volkswagen,Polo


In [ ]:
df.shape

(67423, 10)

In [ ]:
df1=df.drop(columns=["variant"])

In [ ]:
df1["Fuel Type"]=df1["Fuel Type"].fillna("Petrol")
df1["Transmission"]=df1["Transmission"].fillna("Manual")
df1["price"]=df1["price"].replace(",","",regex=True).astype(int)
df1["Kms Driven"]=df1["Kms Driven"].replace(",","",regex=True).astype(int)
df1["Year_since_car_bought"]=2020-df1["Year"]

In [ ]:
d={}
brand=sorted(df1["brand"].unique())
for i in brand:
  models=sorted(df1[df1["brand"]==i]["model"].unique())
  model_transmission_d={}
  for j in models:
    Tranmission=sorted(df1[df1["model"]==j]["Transmission"].unique())
    Fuel=sorted(df1[df1["model"]==j]["Fuel Type"].unique())
    model_transmission_d.update({j:[Tranmission,Fuel]})
  d.update({i:model_transmission_d})

In [ ]:
cities_list=sorted(df1["city"].unique().tolist())

In [ ]:
X=df1.drop(columns=["price","Year"])
y=df1["price"]
cols=X.select_dtypes("object").columns.tolist()
encoder = ce.OrdinalCategoricalEncoder(encoding_method='ordered',variables=cols)
encoder.fit(X, y)
train_t= encoder.transform(X)

In [ ]:
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV
import lightgbm as lgb

In [ ]:
model=lgb.LGBMRegressor()

In [ ]:
kfold=KFold(n_splits=10,shuffle=True,random_state=1)
results=cross_val_score(estimator=model,X=train_t,y=y,cv=kfold,scoring="neg_mean_squared_error",n_jobs=-1)

In [ ]:
params={}
params["max_depth"]=[18]
params["num_leaves"]=[61]
params["colsample_bytree"]=[1]
params["subsample"]=[.9]
params["min_child_samples"]=[16]
params["reg_lambda"]=[.1]
params["learning_rate"]=[.1]
params["n_estimators"]=[400]
params["random_state"]=[1]

In [ ]:
kfold=KFold(n_splits=5,shuffle=True,random_state=1)
grid=GridSearchCV(estimator=model,cv=kfold,param_grid=params,scoring="neg_mean_squared_error",n_jobs=-1,verbose=1)

In [ ]:
grid.fit(train_t,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   34.4s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=N...
                                     subsample_freq=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'colsample_bytree': [1], 'learning_rate': [0.1],
                         'max_depth': [18], 'min_child_samples': [16],
                         'n_estimators': [200, 300, 400, 500],
                         'num_leaves': [61], 'random_state': [1],
         

In [ ]:
grid_df=pd.DataFrame(data=grid.cv_results_)

In [ ]:
grid_df["mean_test_score"]=abs(grid_df["mean_test_score"])**.5
grid_df["std_test_score"]=abs(grid_df["std_test_score"])**.5

In [ ]:
grid_df[["param_n_estimators","mean_test_score","std_test_score"]].sort_values(by="mean_test_score",ascending=True).head(10)

,param_n_estimators,mean_test_score,std_test_score
1,300,153134.167748,87359.952932
2,400,153159.261222,86677.578417
0,200,153253.406524,88037.435530
3,500,153493.610550,86106.613971


In [ ]:
params={}
params["max_depth"]=18
params["num_leaves"]=61
params["colsample_bytree"]=1
params["subsample"]=.9
params["min_child_samples"]=16
params["reg_lambda"]=.1
params["learning_rate"]=.1
params["n_estimators"]=400
params["random_state"]=1

In [ ]:
params

{'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 18,
 'min_child_samples': 16,
 'n_estimators': 400,
 'num_leaves': 61,
 'random_state': 1,
 'reg_lambda': 0.1,
 'subsample': 0.9}

In [ ]:
import pickle

In [ ]:
model=lgb.LGBMRegressor(**params)
model.fit(train_t,y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
              importance_type='split', learning_rate=0.1, max_depth=18,
              min_child_samples=16, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=400, n_jobs=-1, num_leaves=61, objective=None,
              random_state=1, reg_alpha=0.0, reg_lambda=0.1, silent=True,
              subsample=0.9, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
filename = 'finalized_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
filename = 'encoder_category.pkl'
pickle.dump(encoder, open(filename, 'wb'))

In [ ]:
filename = 'dict_models.pkl'
pickle.dump(d, open(filename, 'wb'))

In [ ]:
filename = 'cities_list.pkl'
pickle.dump(cities_list, open(filename, 'wb'))

In [ ]:
files.download("cities_list.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

In [ ]:
files.download("finalized_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("encoder_category.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("dict_models.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>